In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Change this to the location of your wbpc.data file
file_raw_data = "/home/ralampay/workspace/pattern-recognition-course/data/wdbc.csv"

raw_data = pd.read_csv(file_raw_data, header=None)

raw_data

In [ ]:
x = raw_data.iloc[:,2:32]

x

In [ ]:
num_features = len(x.columns)

print("Number of Features: {}".format(num_features))

In [ ]:
columns = []

for i in range(num_features):
    columns.append("x{}".format(i))

x.columns = columns

x

In [ ]:
x_mean = x.mean()
x_std = x.std()
x_standardized = (x - x_mean)/x_std

x_standardized

In [ ]:
x_normalized = (x - x.min()) / (x.max() - x.min())

x_normalized

In [ ]:
y = raw_data[1].replace(['B'], 0).replace(['M'], 1)

y = y.values

y

In [ ]:
num_benign = len(raw_data[raw_data.iloc[:,1] == 'B'])
num_malignant = len(raw_data[raw_data.iloc[:,1] == 'M'])

print("num_benign: {}".format(num_benign))
print("num_malignant: {}".format(num_malignant))

In [ ]:
df = x_normalized.copy()
df['y'] = y

df

In [ ]:
def partition_dataset(df, num_a=20, num_b=20, val_a=1, val_b=0):
    df_a = df[df.iloc[:,-1] == val_a].sample(num_a)
    df_b = df[df.iloc[:,-1] == val_b].sample(num_b)
    
    df.drop(df_a.index, inplace=True)
    df.drop(df_b.index, inplace=True)
    
    frames = [df_a, df_b]
    df_validation = pd.concat(frames)
    
    return df, df_validation

training, validation = partition_dataset(df, num_a=20, num_b=20)

In [ ]:
training

In [ ]:
validation

In [ ]:
x_training = training.iloc[:,:-1].values
x_validation = validation.iloc[:,:-1].values

y_training = training['y'].values
y_validation = validation['y'].values

In [ ]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.input_1 = nn.Linear(input_dim, 25)
        self.input_2 = nn.Linear(25, 20)
        self.input_3 = nn.Linear(20, 10)
        self.output = nn.Linear(10, output_dim)

    def forward(self, x):
        # f(x) = a(f(x))
        x = F.relu(self.input_1(x))
        x = F.relu(self.input_2(x))
        x = F.relu(self.input_3(x))
        y = F.sigmoid(self.output(x))

        return y

In [ ]:
x = torch.FloatTensor(x_training)
model = MultiLayerPerceptron(30, 1)

y = model.forward(x)

y

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
criterion = nn.MSELoss()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
criterion = criterion.to(device)

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        self.n_samples = len(x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [ ]:
x = torch.tensor(x_training).float().to(device)
y = torch.tensor(y_training).float().to(device)

training_ds = CustomDataset(x, y)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 5

train_loader = DataLoader(
    training_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

In [ ]:
def train_fn(loader, model, optimizer, loss_fn, device):
    loop = tqdm(loader)
    
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Forward
        predictions = model.forward(data)
        
        loss = loss_fn(predictions, targets)
        
        # Backward
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        # Update tqdm
        loop.set_postfix(loss=loss.item())

epochs = 100

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    train_fn(train_loader, model, optimizer, criterion, device)

In [ ]:
predictions = model.forward(torch.tensor([x_validation]).float()).cpu().detach().numpy()

predictions = predictions.ravel()

predictions = [1 if y >= 0.5 else 0 for y in predictions]

predictions

In [ ]:
y_validation

In [ ]:
cm = confusion_matrix(y_validation, predictions)
tp = cm[0][0]
tn = cm[1][1]
fp = cm[0][1]
fn = cm[1][0]

mcc = ((tn * tp) - (fn * fp)) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
f1 = tp / (tp + (0.5 * (fp + fn)))

print(classification_report(y_validation, predictions))
print("MCC: {}".format(mcc))
print("F1: {}".format(f1))